In [ ]:
# Ideas to improve accuracy
'''
What to do about cabin no
'''

In [1]:
#import std libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense,Activation

Using Theano backend.


In [37]:
#import data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
train_data.describe()

PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200

In [2]:
#Convert into numpy arrays
numpy_train = train_data.as_matrix()
numpy_test = test_data.as_matrix()
#id ,survived, class, name,sex,age,siblingspouse,parentchild,ticket,fare,cabin,embarked


X_train = numpy_train[:,2:]
# class, name,sex,age,siblingspouse,parentchild,ticket,fare,cabin,embarked
y_train = numpy_train[:,1]
X_test = numpy_test[:,1:]
PassengerId = numpy_test[:,0]

#Removing name from list of parameters as this is irrelevant
X_train = np.delete(X_train,1,axis=1)
# class,sex,age,siblingspouse,parentchild,ticket,fare,cabin,embarked
X_test = np.delete(X_test,1,axis=1)


#Removing cabin from data as less amount of data

X_train = np.delete(X_train,7,axis=1)
# class,sex,age,siblingspouse,parentchild,ticket,fare,embarked
X_test = np.delete(X_test,7,axis=1)

#Removing ticket no  from data as this is irrelevant
X_train = np.delete(X_train,5,axis=1)
X_test = np.delete(X_test,5,axis=1)

#Fare of row 152 is not defined 
#Replace fare with average fare

null_fare = pd.isnull(X_test[:,5])
average_fare = np.mean(X_test[~null_fare][:,5])
X_test[:,5][null_fare] = average_fare

[nan 'C85' nan 'C123' nan nan 'E46' nan nan nan 'G6' 'C103' nan nan nan nan
 nan nan nan nan nan 'D56' nan 'A6' nan nan nan 'C23 C25 C27' nan nan nan
 'B78' nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan 'D33' nan 'B30' 'C52' nan nan nan nan nan 'B28' 'C83' nan nan
 nan 'F33' nan nan nan nan nan nan nan nan 'F G73' nan nan nan nan nan nan
 nan nan nan nan nan nan 'C23 C25 C27' nan nan nan 'E31' nan nan nan 'A5'
 'D10 D12' nan nan nan nan 'D26' nan nan nan nan nan nan nan 'C110' nan nan
 nan nan nan nan nan 'B58 B60' nan nan nan nan 'E101' 'D26' nan nan nan
 'F E69' nan nan nan nan nan nan nan 'D47' 'C123' nan 'B86' nan nan nan nan
 nan nan nan nan 'F2' nan nan 'C2' nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan 'E33' nan nan nan 'B19' nan nan nan 'A7' nan nan 'C49' nan
 nan nan nan nan 'F4' nan 'A32' nan nan nan nan nan nan nan 'F2' 'B4' 'B80'
 nan nan nan nan nan nan nan nan nan 'G6' nan nan nan 'A31' nan nan nan nan
 nan 'D36' nan nan 'D15'

In [3]:
#Analysing data

#Total samples 891
#Featueres removed 3 Remaining features 7 
#Total nan values
#177,687,2
#Age,cabin,embarked are null
'''
pclass 	Ticket class 	1 = 1st, 2 = 2nd, 3 = 3rd
sex 	Sex 	 male/female
Age 	Age in years 	
sibsp 	# of siblings / spouses aboard the Titanic 	
parch 	# of parents / children aboard the Titanic 	
ticket 	Ticket number 	
fare 	Passenger fare 	
cabin 	Cabin number 	
embarked 	Port of Embarkation
'''

'''
Code to check how many nan values are present
print(np.sum(pd.isnull(X_train[:,0])))
print(np.sum(pd.isnull(X_train[:,8])))
'''


#Replacing the male,female with 0 for male 1 for female
male = np.where(X_train[:,1] == 'male')
X_train[:,1][male] = 0

female = np.where(X_train[:,1] == 'female')
X_train[:,1][female] = 1


#Replacing the male,female with 0 for male 1 for female
male = np.where(X_test[:,1] == 'male')
X_test[:,1][male] = 0

female = np.where(X_test[:,1] == 'female')
X_test[:,1][female] = 1


#Understand the age_nan_value
age_null_boolean = pd.isnull(X_train[:,2])
average_age = np.mean(X_train[~age_null_boolean][:,2])
X_train[:,2][age_null_boolean] = average_age


#Understand the age_nan_value
age_null_boolean = pd.isnull(X_test[:,2])
average_age = np.mean(X_test[~age_null_boolean][:,2])
X_test[:,2][age_null_boolean] = average_age


#Understand the cabin_null_values
cabin_null_boolean = pd.isnull(X_train[:,6]) 
value,count = np.unique(X_train[:,6],return_counts=True)
max_value = value[np.argmax(count)]
X_train[:,6][cabin_null_boolean] = max_value
cabin_null_boolean = pd.isnull(X_train[:,6]) 



#Understand the cabin_null_values
cabin_null_boolean = pd.isnull(X_test[:,6]) 
value,count = np.unique(X_test[:,6],return_counts=True)
max_value = value[np.argmax(count)]
X_test[:,6][cabin_null_boolean] = max_value
cabin_null_boolean = pd.isnull(X_test[:,6]) 


#Replacing the cabin values with integers
cclass = np.where(X_train[:,6]=='C')
X_train[:,6][cclass] = 0 
qclass = np.where(X_train[:,6]=='Q')
X_train[:,6][qclass] = 1
sclass = np.where(X_train[:,6]=='S')
X_train[:,6][sclass] = 2


#Replacing the cabin values with integers
cclass = np.where(X_test[:,6]=='C')
X_test[:,6][cclass] = 0 
qclass = np.where(X_test[:,6]=='Q')
X_test[:,6][qclass] = 1
sclass = np.where(X_test[:,6]=='S')
X_test[:,6][sclass] = 2


In [4]:
#Normalise and standarise the train_data
meantrain = np.mean(X_train,axis=0)
meantest = np.mean(X_test,axis=0)
X_train -= meantrain
X_test -= meantest
X_train /= meantrain
X_test /= meantest
print X_train[0]

[0.2994652406417112 -1.0 -0.2592372520474945 0.9120171673819745 -1.0
 -0.7748741404516245 0.30168005843681517]


In [32]:
#Creating a keras model
model = Sequential()
model.add(Dense(891,input_dim=7,activation='relu'))
model.add(Dense(20,activation='relu'))
model.add(Dense(20,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
# model.add(Dense(12,input_dim=7,activation='relu'))
# model.add(Dense(12,activation='relu'))
# model.add(Dense(12,activation='relu'))
# model.add(Dense(1,activation='sigmoid'))

In [33]:
#compiling the model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [34]:
#Fitting the model
model.fit(X_train[:700],y_train[:700],epochs=1000,batch_size=64,verbose=0)

In [35]:
#Predicting the values
prediction = model.predict(X_train[700:])
prediction = [int(round(single)) for single in prediction]
prediction2 = y_train[700:]
totalcorrect = np.sum(prediction == prediction2)
accuracy = totalcorrect/700.00
print accuracy


0.217142857143


In [219]:
#Observation on prediction
#Location value 152 has a nan output
#Analysing input , came to know that one of the columns of 152 was nan

In [220]:
prediction = [int(round(single)) for single in prediction]
data = np.vstack((PassengerId,prediction))
data = np.transpose(data)


In [221]:
dataframe = pd.DataFrame(data)
dataframe.columns = ['PassengerId','Survived']


    PassengerId Survived
0           892        0
1           893        0
2           894        0
3           895        0
4           896        0
5           897        0
6           898        1
7           899        0
8           900        1
9           901        0
10          902        0
11          903        0
12          904        1
13          905        0
14          906        1
15          907        1
16          908        0
17          909        0
18          910        0
19          911        0
20          912        1
21          913        1
22          914        1
23          915        1
24          916        1
25          917        0
26          918        1
27          919        0
28          920        0
29          921        1
..          ...      ...
388        1280        0
389        1281        0
390        1282        0
391        1283        1
392        1284        0
393        1285        0
394        1286        0
395        1287        1


In [222]:
dataframe.to_csv('submit.csv',index=False)